# Recommendation System

## Задание 1

Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). Оставь в датасете только фильмы, которые вышли в "релиз".\
Выведи количество фильмов, оставшихся после фильтрации.

In [1]:
import pandas as pd
df_movies = pd.read_csv('../datasets/tmdb_5000_movies.csv')
df_credit = pd.read_csv('../datasets/tmdb_5000_credits.csv')
df_all = pd.merge(df_movies, df_credit, left_on='id', right_on='movie_id')
df_all = df_all[df_movies.status == 'Released'].copy()
print('Количество фильмов после фильтрации:', df_all.shape[0])

Количество фильмов после фильтрации: 4795


## Задание 2

Самый наивный подход к рекомендации фильмов - рекомендовать фильмы с лучшими оценками пользователей. Фильмы, которые пользуются большей популярностью и признанием критиков, с большей вероятностью понравятся среднему зрителю.

Для справедливой оценки фильмов возьмем текущую рейтинговую систему IMDB (weighted rating (WR)), которая рассчитывается по формуле:
$$WR = \frac{v}{v + m} ⋅ R + \frac{m}{v + m} ⋅ C$$ 
$v$ - количество голосов \
$m$ - количество голосов для включения в финальную таблицу \
$R$ - средняя оценка \
$C$ - средняя оценка всех фильмов 

Имплементируй функцию `weighted_rating`. С её помощью расcчитай рейтинг для каждого фильма и сохрани его в колонку `simple_score`.\
Выведи топ-5 фильмов по получившемуся рейтингу.
> В качестве параметра $m$ выбери 95-й квантиль количества голосов.

In [2]:
C = df_all['vote_average'].mean()
m = df_all['vote_count'].quantile(0.95)

q_movies = df_all.copy().loc[df_all['vote_count'] >= m]

In [3]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R + (m/(m+v)) * C)

In [4]:
q_movies['simple_score'] = q_movies.apply(weighted_rating, axis=1)
q_movies = q_movies.sort_values('simple_score', ascending=False)
q_movies[['title_x', 'vote_count', 'vote_average', 'simple_score']].head(5)

,title_x,vote_count,vote_average,simple_score
1881,The Shawshank Redemption,8205,8.5,7.849025
65,The Dark Knight,12002,8.2,7.773990
662,Fight Club,9413,8.3,7.761012
96,Inception,13752,8.1,7.736496
3232,Pulp Fiction,8428,8.3,7.714726


## Задание 3

Такой подход к рекомендациям очень наивен, так как не учитывает информацию о самом фильме (жанр, режиссер, описание, актеры и т.п). \
**Content Based Filtering** (Фильтрация на основе содержания) - тип рекомендательной системы, которая предлагает пользователям похожие элементы на основе конкретного элемента. Общая идея этих рекомендательных систем заключается в том, что если человеку понравился определенный товар, то ему понравится и похожий на него товар.

<center><img src="../misc/images/content.png" alt= “” width="300" height="500">

Реализуем алгоритм рекомендации на основе описания фильма. Для это требуется провести предобработку текста:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Расcчитай [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильма

Выведи размер получившейся матрицы Tf-Idf

> Для [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) используйте параметры по умолчанию 

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer


vec = TfidfVectorizer(stop_words='english')
df_all['overview'] = df_all['overview'].fillna('')
matrix = vec.fit_transform(df_all.overview)
matrix.shape

(4795, 20970)

## Задание 4

Теперь тебе необходимо вычислить показатель сходства между описаниями фильмов. Используем косинусное расстояние, оно рассчитывается по формуле:
$$cos(Θ) = \frac{A ⋅ B}{∥A∥ ∥B∥} = \frac{ Σ_{i=1}^{n} A_i ⋅ B_i } { \sqrt{Σ_{i=1}^{n}A_{i}^{2}} ⋅ {\sqrt{Σ_{i=1}^{n}B_{i}^{2}}}}$$
Но поскольку мы использовали векторизатор TF-IDF на предыдущем шаге, достаточно вычислить скалярное произведение, которое и даст оценку косинусного сходства. Рассчитать его можно через [linear_kernel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.linear_kernel.html). Результат сохрани в переменную `cosine_sim`.

Выведи размер получившейся матрицы `cosine_sim`.

In [6]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(matrix, matrix)
cosine_sim.shape

(4795, 4795)

## Задание 5

Напиши функцию `get_recommendations`. На вход она принимает:
* `movies_dataset` - датасет фильмов
* `title` - название фильма, для которого мы будем искать похожие
* `cosine_sim` - матрица расстояний между описаниями
* `top_k` - топ-k cхожих фильмов

Возвращает top_k названий фильмов, описание которых похоже на выбранный фильм.\
Выведи топ-5 фильмов для `title='Saving Private Ryan'`

In [7]:
def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):
    """get_recommendations"""
    indices = pd.Series(
        movies_dataset.index, index=movies_dataset['original_title']).drop_duplicates()

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_k+1]
    movie_ind = [i[0] for i in sim_scores]
    return list(df_all['original_title'].iloc[movie_ind])

In [8]:
get_recommendations(movies_dataset=df_all,
                    title='Saving Private Ryan', cosine_sim=cosine_sim, top_k=5)

['The Great Raid',
 'The Monuments Men',
 'The Expendables 2',
 'Abandoned',
 'The Train']

## Задание 6

Еще один подход к построению рекомендательной системы - подход на основе сходства между пользователями. Этот подход называется **Collaborative Filtering** (Коллаборативная фильтрация).
<center><img src="../misc/images/all.png" alt= “” width="600" height="700"></center>
Коллаборативная фильтрация - это тип рекомендательной системы, которая использует поведение и предпочтения похожих пользователей для рекомендации товаров или продуктов конкретному пользователю. Система собирает данные о прошлом поведении пользователей, такие как покупки, рейтинги и отзывы, и анализирует их для выявления закономерностей и сходства между пользователями. На основе этих закономерностей система рекомендует товары, которые понравились или были приобретены другими такими же пользователями в прошлом.

Для реализации Коллаборативной фильтрации нам потребуются оценки пользователей [ratings](../datasets/ratings.csv).

>userId - id пользователя \
movieId - id фильма \
rating - оценка фильма (от 0 до 5)\
timestamp - время оценки


Воспользуйся библиотекой [surprise](https://surpriselib.com/) для обучения модели оценки рейтинга фильма [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD). Выведи средние значения 'RMSE', 'MAE' на кросс-валидации с параметрами `cv=5`.

In [9]:
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import SVD
df_ratings = pd.read_csv('../datasets/ratings.csv')

reader = Reader(rating_scale=(1, 5))
df1 = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']], reader)

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, df1,
               measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8968  0.8913  0.9020  0.8985  0.9003  0.8978  0.0037  
MAE (testset)     0.6890  0.6854  0.6943  0.6931  0.6924  0.6908  0.0032  
Fit time          1.33    1.36    1.43    1.40    1.37    1.38    0.03    
Test time         0.29    0.19    0.23    0.29    0.21    0.24    0.04    
